<a href="https://colab.research.google.com/github/maideas/numpy-neural-network/blob/master/examples/inception_mnist/inception_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive

drive.mount('/content/drive')

!ls /content/drive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
'My Drive'


In [0]:
cd /content

/content


In [0]:
!rm -rf numpy-neural-network
!git clone https://github.com/maideas/numpy-neural-network.git

Cloning into 'numpy-neural-network'...
remote: Enumerating objects: 365, done.
remote: Counting objects: 100% (365/365), done.
remote: Compressing objects: 100% (227/227), done.
remote: Total 886 (delta 203), reused 266 (delta 135), pack-reused 521
Receiving objects: 100% (886/886), 68.43 MiB | 42.85 MiB/s, done.
Resolving deltas: 100% (494/494), done.


In [0]:
cd /content/numpy-neural-network/examples/inception_mnist/

/content/numpy-neural-network/examples/inception_mnist


In [0]:
!mkdir -p png png_2

In [0]:
import sys

sys.path.append("../../")

In [0]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import numpy_neural_network as npnn
import npnn_datasets

In [0]:
model = npnn.network.Model([
    npnn.Inception((28, 28, 1),
           2,
        3, 6,
        3, 6,
           2
    ),
    npnn.MaxPool(shape_in=(28, 28, 16), shape_out=(14, 14, 16), kernel_size=2),
    npnn.Inception((14, 14, 16),
            4,
        6,  8,
        6,  8,
            2
    ),
    npnn.MaxPool(shape_in=(14, 14, 22), shape_out=(7, 7, 22), kernel_size=2),
    npnn.Inception((7, 7, 22),
           4,
        8, 8,
        8, 8,
           2
    ),
    npnn.Dense(7*7*22, 140),
    npnn.LeakyReLU(140),
    npnn.Dense(140, 10),
    npnn.Softmax(10)
])

In [0]:
model.loss_layer = npnn.loss_layer.CrossEntropyLoss(10)
optimizer = npnn.optimizer.Adam(model, alpha=1e-3)
optimizer.dataset = npnn_datasets.MNIST()

In [0]:
plt.ion()
plt.show()
fig1, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(10,8))
fig2, axs = plt.subplots(4, 3, figsize=(10,8))

episodes = []

mini_train_loss = []
mini_validation_loss = []
mini_train_accuracy = []
mini_validation_accuracy = []

for episode in np.arange(400):

    # step the optimizer ...
    optimizer.step()
    episodes.append(episode)

    #===========================================================================
    # mini batch loss and accuracy
    #===========================================================================

    tloss = np.mean(optimizer.loss)
    taccuracy = optimizer.accuracy

    mini_train_loss.append(tloss)
    mini_train_accuracy.append(taccuracy)

    ax1.cla()
    ax1.set_xlabel('episode')
    ax1.set_ylabel('train mini-batch loss')
    ax1.set_yscale('log')
    ax1.set_ylim((min(mini_train_loss)/2.0, max(mini_train_loss)*2.0))
    ax1.plot(episodes, mini_train_loss)

    ax2.cla()
    ax2.set_xlabel('episode')
    ax2.set_ylabel('train mini-batch accuracy [%]')
    ax2.set_ylim(-5, 105)
    ax2.plot(episodes, mini_train_accuracy)

    #===========================================================================
    # complete dataset loss and accuracy
    #===========================================================================

    x_validation_batch, t_validation_batch, _ = optimizer.dataset.get_validation_batch()
    y_validation_batch = optimizer.predict(x_validation_batch, t_validation_batch)

    vloss = np.mean(optimizer.loss)
    vaccuracy = optimizer.accuracy

    mini_validation_loss.append(vloss)
    mini_validation_accuracy.append(vaccuracy)

    # print the episode and loss values ...
    print("episode = {0}, tloss = {1}, vloss = {2}, taccuracy = {3}, vaccuracy = {4}".format(
        episode, tloss, vloss, taccuracy, vaccuracy
    ))

    ax3.cla()
    ax3.set_xlabel('episode')
    ax3.set_ylabel('validation mini-batch loss')
    ax3.set_yscale('log')
    ax3.set_ylim((min(mini_validation_loss)/2.0, max(mini_validation_loss)*2.0))
    ax3.plot(episodes, mini_validation_loss, c='tab:orange')

    ax4.cla()
    ax4.set_xlabel('episode')
    ax4.set_ylabel('validation mini-batch accuracy [%]')
    ax4.set_ylim(-5, 105)
    ax4.plot(episodes, mini_validation_accuracy, c='tab:orange')

    #===========================================================================
    # network output plots (validation batch based)
    #===========================================================================

    k = np.arange(x_validation_batch.shape[0])

    for n in np.arange(4):
        for m in np.arange(3):
            c = n * 3 + m
            if c < 10:
                axs[n, m].cla()
                axs[n, m].set_ylim(-0.1, 1.1)
                axs[n, m].scatter(k, t_validation_batch[:,c], s=10, c='tab:green')
                axs[n, m].scatter(k, y_validation_batch[:,c], s=10, c='tab:orange')

    #===========================================================================
    # draw and save PNG to generate video files later on
    #===========================================================================

    plt.draw()
    fig1.savefig('png/episode{0:04d}.png'.format(episode))
    fig2.savefig('png_2/episode{0:04d}.png'.format(episode))
    plt.pause(0.001)

episode = 0, tloss = 0.2450249920340088, vloss = 0.22640067112796372, taccuracy = 6.862745098039215, vaccuracy = 15.686274509803921


episode = 1, tloss = 0.22896003633472492, vloss = 0.212471655020879, taccuracy = 13.235294117647058, vaccuracy = 31.372549019607842


<Figure size 432x288 with 0 Axes>

episode = 2, tloss = 0.21276280638210587, vloss = 0.21013494769153404, taccuracy = 24.50980392156863, vaccuracy = 44.6078431372549


<Figure size 432x288 with 0 Axes>

...

<Figure size 432x288 with 0 Axes>

episode = 398, tloss = 0.03202233971426083, vloss = 0.03818459020599111, taccuracy = 88.23529411764706, vaccuracy = 87.25490196078431


<Figure size 432x288 with 0 Axes>

episode = 399, tloss = 0.03765760863736635, vloss = 0.0455955280150651, taccuracy = 85.29411764705883, vaccuracy = 85.7843137254902


<Figure size 432x288 with 0 Axes>

In [0]:
!zip -r png.zip png png_2

  adding: png/ (stored 0%)
  adding: png/episode0346.png (deflated 5%)
  adding: png/episode0050.png (deflated 7%)
  adding: png/episode0111.png (deflated 5%)
  adding: png/episode0126.png (deflated 6%)
  adding: png/episode0286.png (deflated 5%)
  adding: png/episode0105.png (deflated 5%)
  adding: png/episode0296.png (deflated 5%)
  adding: png/episode0096.png (deflated 5%)
  adding: png/episode0207.png (deflated 5%)
  adding: png/episode0375.png (deflated 5%)
  adding: png/episode0190.png (deflated 5%)
  adding: png/episode0222.png (deflated 5%)
  adding: png/episode0225.png (deflated 5%)
  adding: png/episode0384.png (deflated 5%)
  adding: png/episode0024.png (deflated 8%)
  adding: png/episode0281.png (deflated 5%)
  adding: png/episode0217.png (deflated 5%)
  adding: png/episode0187.png (deflated 5%)
  adding: png/episode0378.png (deflated 5%)
  adding: png/episode0106.png (deflated 6%)
  adding: png/episode0040.png (deflated 7%)
  adding: png/episode0244.png (deflated 5%)
  add

In [0]:
!cp -f png.zip /content/drive/'My Drive'/'Colab Notebooks'/